# Create a manifest file

The *manifest.txt* file created here is intended to be used with [gdc-client](https://gdc.cancer.gov/access-data/gdc-data-transfer-tool)

Please refer to [gdc API](https://docs.gdc.cancer.gov/API/Users_Guide/Getting_Started/) for reference about query done here

In [1]:
%load_ext watermark
%watermark -v -m  -u -n -p pandas,requests,json -a Filippo_Valle -g -r -b -w

Filippo_Valle 
last updated: Fri Oct 16 2020 

CPython 3.8.5
IPython 7.18.1

pandas 1.1.2
requests 2.24.0
json 2.0.9

compiler   : GCC 7.5.0
system     : Linux
release    : 5.4.0-48-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 12
interpreter: 64bit
Git hash   : 30429f731810f277797e7fc4311426bd10484579
Git repo   : git@github.com:fvalle1/cancers
Git branch : develop
watermark 2.0.2


In [2]:
import requests
import json
import pandas as pd

In [3]:
# use file endopint
files_endpt = "https://api.gdc.cancer.gov/files"

# The 'fields' parameter is passed as a comma-separated string of single names.
fields = [
    "file_name",
    #"analysis.workflow_type",
    #"experimental_strategy",
    "cases.project.primary_site",
    "cases.project.disease_type",
    "cases.project.project_id",
    "cases.submitter_id",
    "cases.samples.portions.analytes.aliquots.submitter_id",
    "cases.demographic.vital_status",
    "cases.demographic.days_to_birth",
    "cases.demographic.days_to_death",
    "cases.demographic.gender",
    "cases.diagnoses.primary_diagnosis",
    "cases.diagnoses.tumor_stage",
    "cases.diagnoses.tumor_grade",
    "cases.diagnoses.last_known_disease_status",
    "cases.diagnoses.progression_or_recurrence"
    "cases.diagnoses.days_to_last_follow_up",
    "cases.diagnoses.age_at_diagnosis",
    "cases.diagnoses.morphology",
    "cases.diagnoses.tissue_or_organ_of_origin",
    "cases.exposures.years_smoked",
    "cases.exposures.cigarettes_per_day",
    "cases.samples.longest_dimension",
    "samples.portions.analytes.aliquots.submitter_id",
]

fields = ','.join(fields)

Filter gene expression data obtained with *RNA-Seq HTSeq* in txt format either from **BRCA** or **LUAD** and **LUSC**

Don't forget to select the correct filter for *cases.project.project_id*

In [4]:
# Tissue project filters
filters = {
    "op": "and",
    "content":[
        {
        "op": "in",
        "content":{
            "field": "files.data_type",
            "value": ["Gene Expression Quantification"]
            }
        },
        {
        "op": "in",
        "content":{
               "field": "files.analysis.workflow_type",
                "value": ["HTSeq - FPKM"]
                }
        },
        {
        "op": "in",
        "content":{
            "field": "files.data_format",
            "value": ["TXT"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "cases.project.project_id",
            "value": ['TCGA-LUAD', 'TCGA-LUSC'] #<-- uncomment this line for Lung
            #"value": ["TCGA-BRCA"] #<-- uncomment this line for Breast
            }
        }
        
    ]
}

In [5]:
# A POST is used, so the filter parameters can be passed directly as a Dict object.
##remove manifest to have file fields
params = {
   "return_type": "manifest",
    "filters": filters,
    "fields": fields,
    "format": "TSV",
    "size": "15000"
    }

In [6]:
# The parameters are passed to 'json' rather than 'params' in this case
response = requests.post(files_endpt, headers = {"Content-Type": "application/json"}, json = params)

#print(response.content.decode("utf-8"))

In [7]:
with open("manifest.txt","w") as manifest:
    manifest.write(response.content.decode("utf-8"))

# Files

Write a file with all metadata, useful in next analyses

In [8]:
params = {
    "filters": json.dumps(filters),
    "fields": fields,
    "format": "TSV",
    "size": "15000"
    }
response = requests.post(files_endpt, headers = {"Content-Type": "application/json"}, json = params)
with open("files.txt","w") as files:
    files.write(response.content.decode("utf-8"))

In [9]:
df_files = pd.read_csv("files.txt", sep='\t')
df_files.drop("id", axis=1, inplace=True)
df_files.set_index("file_name", inplace=True)
df_files.head(3)

,cases.0.demographic.days_to_birth,cases.0.demographic.days_to_death,cases.0.demographic.gender,cases.0.demographic.vital_status,cases.0.diagnoses.0.age_at_diagnosis,cases.0.diagnoses.0.last_known_disease_status,cases.0.diagnoses.0.morphology,cases.0.diagnoses.0.primary_diagnosis,cases.0.diagnoses.0.tissue_or_organ_of_origin,cases.0.diagnoses.0.tumor_grade,cases.0.diagnoses.0.tumor_stage,cases.0.exposures.0.cigarettes_per_day,cases.0.exposures.0.years_smoked,cases.0.project.disease_type,cases.0.project.primary_site,cases.0.project.project_id,cases.0.samples.0.longest_dimension,cases.0.samples.0.portions.0.analytes.0.aliquots.0.submitter_id,cases.0.submitter_id
file_name,,,,,,,,,,,,,,,,,,,
fadfa0b7-8543-4ca3-8c26-9114e71caaeb.FPKM.txt.gz,-25931.0,NaN,female,Alive,25931.0,not reported,8140/3,"Adenocarcinoma, NOS","Middle lobe, lung",not reported,not reported,1.095890,20.0,Lung Adenocarcinoma,Lung,TCGA-LUAD,1.0,TCGA-64-1678-01A-01R-0946-07,TCGA-64-1678
41059154-b4e8-4b61-9600-8b19262ccab0.FPKM.txt.gz,-19855.0,NaN,female,Alive,19855.0,not reported,8140/3,"Adenocarcinoma, NOS","Upper lobe, lung",not reported,stage ib,2.958904,36.0,Lung Adenocarcinoma,Lung,TCGA-LUAD,1.1,TCGA-44-8117-01A-11R-2241-07,TCGA-44-8117
1dc8075b-ed0c-471f-8947-967d23ff686b.FPKM.txt.gz,-24014.0,NaN,male,Alive,24014.0,not reported,8140/3,"Adenocarcinoma, NOS","Upper lobe, lung",not reported,stage ib,3.068493,28.0,Lung Adenocarcinoma,Lung,TCGA-LUAD,1.0,TCGA-44-2662-01A-01R-0946-07,TCGA-44-2662


In [10]:
# save file to dat
df_files.to_csv("files_tcga.dat", header=True)